In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import classification_report
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('avocado.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop(columns=['Unnamed: 0','Date'],axis=1)

In [ ]:
df

In [ ]:
df.info()

We can see there are no null values, so imputation is not required.

Theres 2 features with object data type (Type, Region), so we need to use encoding for them, as well as for feature Year, as it can have effect on Price but need not be in YYYY format

In [ ]:
print(df['type'].value_counts().sort_values())

In [ ]:
print(df['region'].value_counts().sort_values())

Using LabelEncoder for both of them would be best

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab_enc=LabelEncoder()

In [ ]:
data=lab_enc.fit_transform(df['region'])
pd.Series(data)

In [ ]:
df['region']=data
df

In [ ]:
data=lab_enc.fit_transform(df['type'])
pd.Series(data)

In [ ]:
df['type']=data
df

In [ ]:
data=lab_enc.fit_transform(df['year'])
pd.Series(data)

In [ ]:
df['year']=data
df

In [ ]:
plt.figure(figsize=(20,15),facecolor='yellow')
plotno=1
for column in df:
    if plotno<=13:
        ax=plt.subplot(3,5,plotno)
        sns.distplot(df[column])
        plt.xlabel(column,fontsize=20)
    plotno+=1
plt.tight_layout()

# Case 1 - Find the region

In [ ]:
y=df['region']
x=df.drop(columns=['region'])

In [ ]:
plt.figure(figsize=(20,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=18:
        ax=plt.subplot(5,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('region',fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
df_corr=df.corr().abs()
plt.figure(figsize=(15,11))
sns.heatmap(df_corr,annot=True,annot_kws={'size':10})
plt.show

From heatmap we can see region has weak bond with Year, type

In [ ]:
plt.figure(figsize=(15,15))
plotno=1
for column in x:
    if plotno<=13:
        ax=plt.subplot(4,4,plotno)
        sns.stripplot(y,x[column])
    plotno+=1
plt.show()

In [ ]:
df_features=df
plt.figure(figsize=(20,20),facecolor='red')
plotno=1
for column in df_features:
    if plotno<=18:
        ax=plt.subplot(5,5,plotno)
        sns.boxplot(data=df_features[column])
        plt.xlabel(column,fontsize=12)
    plotno+=1
plt.show()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
q1=df.quantile(0.25)
q3=df.quantile(0.75)
iqr=q3-q1

In [ ]:
AvgPrice=(q3.AveragePrice + (1.5*iqr.AveragePrice))
index=np.where(df['AveragePrice']>AvgPrice)

In [ ]:
df=df.drop(df.index[index])
df.shape

In [ ]:
df_features=df
plt.figure(figsize=(20,20),facecolor='red')
plotno=1
for column in df_features:
    if plotno<=18:
        ax=plt.subplot(5,5,plotno)
        sns.boxplot(data=df_features[column])
        plt.xlabel(column,fontsize=12)
    plotno+=1
plt.show()

For other features, the outliers consist lot of data and removal would harm the dataset

In [ ]:
df.reset_index()

In [ ]:
sns.lineplot(x='region',y='Total Volume',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='AveragePrice',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='4046',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='4225',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='4770',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='Total Bags',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='Small Bags',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='Large Bags',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='XLarge Bags',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='type',data=df)
plt.show()

In [ ]:
sns.lineplot(x='region',y='year',data=df)
plt.show()

From these graphs and the heatplot we can see Year and Type have very little relation with region, so they can be excluded from prediction model

In [ ]:
y=df['region']
x=df.drop(columns=['region','year','type'])

In [ ]:
from sklearn.feature_selection import SelectKBest,f_classif

In [ ]:
best_features=SelectKBest(score_func=f_classif,k=9)

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(7,'Score'))

Based om the lineplot, heatmap, and Selectk method, these 7 features are best syited to make prediction for region

In [ ]:
y=df['region']
x=df[['Total Volume','4225','Small Bags','4046','Total Bags','Large Bags','4770']]

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [ ]:
vif=pd.DataFrame()
vif['vif']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif['Features']=x.columns
vif

Considering most used multicollinearity threshold of 10, we're clear of multicollinearity

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.17,random_state=333)

In [ ]:
def metric_score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        y_pred=clf.predict(x_train)
        print('\n----Train result----')
        print(f'Accuracy Score:{accuracy_score(y_train,y_pred)*100:.2f}%')
        
    elif train==False:
        pred=clf.predict(x_test)
        print('\n----Test result----')
        print(f'Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%')

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)

In [ ]:
metric_score(knn,x_train,x_test,y_train,y_test,train=True)
metric_score(knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
metric_score(log_reg,x_train,x_test,y_train,y_test,train=True)
metric_score(log_reg,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier(n_neighbors=9),
                          n_estimators=6,max_samples=0.75,
                          bootstrap=True,random_state=366,oob_score=True)

In [ ]:
bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

Bagging did not improve accuracy score.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor

In [ ]:
rnd_clf=RandomForestClassifier()
rnd_clf.fit(x_train,y_train)

In [ ]:
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=True)
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=False)

In [ ]:
pickle.dump(reg,open('knn','wb'))

Random Forest is giving best results, so we'll use this for Prediction

In [ ]:
print('Region:',rnd_clf.predict(scaler.transform([[17074.83,1529.20,13066.82,2046.96,13498.67,431.85,0.00]])))

# Case 2 - Find Average Price

In [ ]:
y=df['AveragePrice']
x=df.drop(columns=['AveragePrice'])

In [ ]:
plt.figure(figsize=(20,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=18:
        ax=plt.subplot(5,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('AveragePrice',fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
df_corr=df.corr().abs()
plt.figure(figsize=(15,11))
sns.heatmap(df_corr,annot=True,annot_kws={'size':10})
plt.show

From Scatterplot and heatmap, we can see region and year have weak bond with Price.

We can also see Type has strongest bond

In [ ]:
x

In [ ]:
sns.lineplot(x='AveragePrice',y='region',data=df)
plt.show()

In [ ]:
sns.lineplot(x='AveragePrice',y='Total Volume',data=df)
plt.show()

In [ ]:
sns.lineplot(x='AveragePrice',y='4046',data=df)
plt.show()

In [ ]:
sns.lineplot(x='AveragePrice',y='4225',data=df)
plt.show()

In [ ]:
sns.lineplot(x='AveragePrice',y='type',data=df)
plt.show()

In [ ]:
sns.lineplot(x='AveragePrice',y='year',data=df)
plt.show()

In [ ]:
y=df['AveragePrice']
x=df.drop(columns=['AveragePrice','year','region'])

In [ ]:
scal=StandardScaler()
x_scaled=scal.fit_transform(x)

In [ ]:
vif=pd.DataFrame()
vif['vif']=[variance_inflation_factor(x_scaled,i) for i in range(x_scaled.shape[1])]
vif['Features']=x.columns
vif

Threshold is in control

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
bestf=SelectKBest(score_func=f_classif,k=9)
fit=bestf.fit(x,y)
df_scores=pd.DataFrame(fit.scores_)
df_columns=pd.DataFrame(x.columns)

In [ ]:
feat_score=pd.concat([df_columns,df_scores],axis=1)
feat_score.columns=['Feature_Name','score']
print(feat_score.nlargest(7,'score'))

In [ ]:
y=df[['AveragePrice']]
x=df[['type','4046','Total Volume','4770','4225','Total Bags','Small Bags']]

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.12,random_state=44)

In [ ]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso,Ridge,RidgeCV,LassoCV

In [ ]:
lasscv=LassoCV(alphas=None,max_iter=200,normalize=True)
lasscv.fit(x_train,y_train)

In [ ]:
alpha=lasscv.alpha_
alpha

In [ ]:
lasso_reg=Lasso(alpha)
lasso_reg.fit(x_train,y_train)

In [ ]:
lasso_reg.score(x_test,y_test)

In [ ]:
ridgecv=RidgeCV(alphas=np.arange(0.002,0.1,0.02),normalize=True)
ridgecv.fit(x_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_model=Ridge(alpha=ridgecv.alpha_)
ridge_model.fit(x_train,y_train)

In [ ]:
ridge_model.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_param={'criterion':['gini','entropy'],
           'max_depth':range(10,15),
           'min_samples_leaf':range(2,6),
           'min_samples_split':range(3,8),
           'max_leaf_nodes':range(5,10)}

In [ ]:
grid_search = GridSearchCV(estimator=reg,
                           param_grid=grid_param,
                           cv=5,
                           n_jobs=-1)

In [ ]:
pickle.dump(reg,open('Avocado','wb'))